In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
processed_data_dir = 'processed_data'

In [27]:
import os
em_downsampled_transformed_name = 'dp1_em_downsampled_stack_transformed'
em_affine3d_file = os.path.join(processed_data_dir, f'transformed_stacks/{em_downsampled_transformed_name}_affine3d_mat.txt')

landmark_dir = 'landmarks'
landmark_base_name = 'landmarks_202404020045'
landmark_file = os.path.join(landmark_dir, f'{landmark_base_name}.csv')


In [ ]:
import pandas as pd
landmarkdf = pd.read_csv(landmark_file)
# Rename x_EM, y_EM, z_EM to x_em_clemui, y_em_clemui, z_em_clemui
# and x_LM, y_LM, z_LM to x_lm_clemui, y_lm_clemui, z_lm_clemui
landmarkdf.rename(columns={
    'x_EM': 'x_em_clemui',
    'y_EM': 'y_em_clemui',
    'z_EM': 'z_em_clemui',
    'x_LM': 'x_lm_clemui',
    'y_LM': 'y_lm_clemui',
    'z_LM': 'z_lm_clemui'
}, inplace=True)
landmarkdf

,landmark_name,tag,x_EM,y_EM,z_EM,x_LM,y_LM,z_LM
0,Pt-2,confirmed,1392.065162,939.549477,89.762090,1368.949598,938.602359,100.0
1,Pt-3,confirmed,1698.571514,874.969650,85.243414,1643.211884,868.415943,100.0
2,Pt-4,confirmed,1410.085755,925.032202,470.814911,1407.837509,920.639177,500.0
3,Pt-5,confirmed,1414.312151,731.356375,309.191181,1415.024726,727.486650,300.0
4,Pt-6,confirmed,1470.753244,621.977165,313.577356,1481.816907,622.103430,300.0
...,...,...,...,...,...,...,...,...
752,landmark-1646,confirmed,502.950788,861.809945,701.000000,491.789428,851.392675,701.0
753,landmark-1647,confirmed,805.435878,515.025374,701.000000,804.305056,515.025374,701.0
754,landmark-1648,confirmed,1018.603208,662.355362,695.000000,1026.788205,654.170365,699.0
755,landmark-1649,confirmed,1624.355391,859.196862,699.000000,1657.378081,873.873613,699.0


In [29]:
# Check if landmark_name in landmarkdf is unique
landmarkdf['landmark_name'].is_unique

True

In [ ]:
# Convert EM to {em_transformed_name}
landmarkdf[f'x_{em_downsampled_transformed_name}'] = landmarkdf['x_em_clemui']
landmarkdf[f'y_{em_downsampled_transformed_name}'] = landmarkdf['y_em_clemui']
landmarkdf[f'z_{em_downsampled_transformed_name}'] = landmarkdf['z_em_clemui']

In [ ]:
# Convert LM to {lm_name}
lm_name = 'dp1_lm_anatomy_trial11_CLAHE'
lm_scale_zyx = (100, 4, 4)  # scale z, y, x

landmarkdf[f'x_{lm_name}'] = (landmarkdf['x_em_clemui'] / lm_scale_zyx[2]).round().astype(int)
landmarkdf[f'y_{lm_name}'] = (landmarkdf['y_em_clemui'] / lm_scale_zyx[1]).round().astype(int)
landmarkdf[f'z_{lm_name}'] = (landmarkdf['z_em_clemui'] / lm_scale_zyx[0]).round().astype(int)

In [ ]:
import numpy as np
from compute_affine3d_from_landmarks import invert_affine_matrix, transform_points

# Convert {em_downsampled_transformed_name} to {em_downsampled_name} by applying affine3d
em_downsampled_name = 'dp1_em_downsampled_stack'
affine3d_mat = np.loadtxt(em_affine3d_file)
inv_affine3d_mat = invert_affine_matrix(affine3d_mat)

points = landmarkdf[[f'z_{em_downsampled_transformed_name}', f'y_{em_downsampled_transformed_name}', f'x_{em_downsampled_transformed_name}']].values
# Apply affine3d_mat to points
inv_transformed_points = transform_points(points, inv_affine3d_mat)
inv_transformed_points

landmarkdf[f'x_{em_downsampled_name}'] = inv_transformed_points[:, 2]
landmarkdf[f'y_{em_downsampled_name}'] = inv_transformed_points[:, 1]
landmarkdf[f'z_{em_downsampled_name}'] = inv_transformed_points[:, 0]

x_dp1_em_downsampled_stack    1448.957708
y_dp1_em_downsampled_stack    1189.759887
z_dp1_em_downsampled_stack    1583.895026
dtype: float64

In [ ]:
# Convert {em_downsampled_name} to neuroglancer
em_downsample_scale_zyx = (176, 176, 200) # scale z, y, x
neuroglancer_scale_ngxyz = (11, 11, 25) # Neuroglancer voxel size in nm in neuroglancer coordinates x, y, z

# Note that the order of x, y, z is different in neuroglancer (due to permutations of axes caused in brainmaps export + imagej import+ imagej export, historical reasons, and non-careful handling of downsampled and meta data)
# the unit is in pixel
landmarkdf[f'x_neuroglancer_px'] = (landmarkdf[f'z_{em_downsampled_name}'] * em_downsample_scale_zyx[0] / neuroglancer_scale_ngxyz[0]).round().astype(int)
landmarkdf[f'y_neuroglancer_px'] = (landmarkdf[f'y_{em_downsampled_name}'] * em_downsample_scale_zyx[1] / neuroglancer_scale_ngxyz[1]).round().astype(int)
landmarkdf[f'z_neuroglancer_px'] = (landmarkdf[f'x_{em_downsampled_name}'] * em_downsample_scale_zyx[2] / neuroglancer_scale_ngxyz[2]).round().astype(int)


In [34]:
# Save the full landmarkdf to a new csv file
landmarkdf.to_csv(os.path.join(landmark_dir, f'{landmark_base_name}_lm_to_neuroglancer.csv'), index=False)

In [35]:
# Generate json file
# Template json file
ng_json_dir = 'neuroglancer_json'
template_json_file = os.path.join(ng_json_dir, 'from_Nila_221010_151223__dp_somata_.json')
import json
with open(template_json_file, 'r') as f:
    template_json = json.load(f)

In [ ]:
template_json['layers'][1]['annotations']

annotations = []
for _, row in landmarkdf.iterrows():
    # Get the coordinates
    x = row[f'x_neuroglancer_px']
    y = row[f'y_neuroglancer_px']
    z = row[f'z_neuroglancer_px']
    # Get the name
    name = row['landmark_name']
    # Get the radius
    #radius = [250, 250, 110]  # in nm
    radius = [100, 100, 44]  # in nm
    # Create the annotation
    annotation = {
        'center': [x, y, z],
        'radii': radius,
        'type': 'ellipsoid',
        'id': name,
    }
    annotations.append(annotation)
# Replace the annotations to the template json
template_json['layers'][1]['annotations'] = annotations

# Save the json file
import json
json_file = os.path.join(ng_json_dir, f'{landmark_base_name}_lm_to_neuroglancer.json')
with open(json_file, 'w') as f:
    json.dump(template_json, f, indent=4)